In [1]:
import time
start = time.time()

In [2]:
#Przygotowanie danych
import numpy as np
import pickle as pkl
from sklearn.metrics import confusion_matrix
from sklearn.datasets import fetch_openml
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

In [3]:
mnist = fetch_openml('mnist_784', version=1)

In [4]:
X = mnist['data']
y = mnist['target'].astype(np.uint8)

In [5]:
print(X)

       pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
69995     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
69996     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
69997     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
69998     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
69999     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

       pixel10  ...  pixel7

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 784 entries, pixel1 to pixel784
dtypes: float64(784)
memory usage: 418.7 MB


In [7]:
print(y)

0        5
1        0
2        4
3        1
4        9
        ..
69995    2
69996    3
69997    4
69998    5
69999    6
Name: class, Length: 70000, dtype: uint8


In [8]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 70000 entries, 0 to 69999
Series name: class
Non-Null Count  Dtype
--------------  -----
70000 non-null  uint8
dtypes: uint8(1)
memory usage: 68.5 KB


In [9]:
print((np.array(mnist.data.loc[42]).reshape(28, 28) > 0).astype(int))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 

In [10]:
#Zbiór uczący i testowy

In [11]:
#Posortowanie danych
y = y.sort_values()
index = y.index
X = X.reindex(index=index)

In [12]:
#losowy podział
shuffle_index = np.random.permutation(70000)
X = X.reindex(index=shuffle_index)
y = y.reindex(index=shuffle_index)

In [13]:
X_train, X_test = X[:56000], X[56000:]
y_train, y_test = y[:56000], y[56000:]
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(56000, 784) (56000,)
(14000, 784) (14000,)


In [14]:
y_train.values

array([2, 9, 3, ..., 1, 5, 2], dtype=uint8)

In [15]:
y_test.values

array([4, 8, 5, ..., 7, 0, 6], dtype=uint8)

In [16]:
#Uczenie - jedna klasa

In [17]:
y_train_0 = (y_train == 0)
y_test_0 = (y_test == 0)
print(y_train_0)
print(np.unique(y_train_0))
print(len(y_train_0))

47862    False
51913    False
32975    False
39449    False
52092    False
         ...  
38511    False
27270    False
5245     False
23918    False
39545    False
Name: class, Length: 56000, dtype: bool
[False  True]
56000


In [18]:
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_0)

SGDClassifier(random_state=42)

In [19]:
#4 - policz dokładność (accurancy)

In [20]:
scoreTrain = sgd_clf.score(X_train,y_train_0)
scoreTest = sgd_clf.score(X_test,y_test_0)

acc = list()
acc.append(scoreTrain)
acc.append(scoreTest)

print(scoreTrain, scoreTest)
fileObject = open("sgd_acc.pkl", 'wb')
pkl.dump(acc, fileObject)
fileObject.close()

0.9888214285714285 0.9865714285714285


In [21]:
#4 - walidacja krzyżowa dokładności modelu

In [22]:
score = cross_val_score(sgd_clf, X_train, y_train_0,
cv=3, scoring="accuracy")

In [23]:
print(score)
fileObject = open("sgd_cva.pkl", 'wb')
pkl.dump(score, fileObject)
fileObject.close()

[0.9858574  0.98832164 0.97064181]


In [24]:
#Uczenie - wiele klas

In [25]:
sgd_clf.fit(X_train, y_train)

SGDClassifier(random_state=42)

In [26]:
score = cross_val_score(sgd_clf, X_train, y_train,
cv=3, scoring="accuracy")
print(score)

[0.86805593 0.87089516 0.88085289]


In [27]:
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train, cv=3)

In [28]:
conf_mx = confusion_matrix(y_train, y_train_pred)
print(conf_mx)

[[5270    0   34   14   11    9   63    6   81   18]
 [   3 5955   77   23    8   13   23   23  168   14]
 [  42   36 4924   97   64   26  139   71  158   31]
 [  21   17  235 4860   27   88   67   81  250   75]
 [   7   19   48    8 4965    9   97   53   84  239]
 [  77   18   70  377  105 3683  188   27  346   93]
 [  40    8   91    3   37   53 5229    5   35    1]
 [  25   20  109   10   67   10    7 5450   28  134]
 [  44  102  200  171   67  108   65   58 4565  123]
 [  27   17   48   87  564   36    5  542  172 4002]]


In [29]:
print(type(conf_mx))

<class 'numpy.ndarray'>


In [30]:
fileObject = open("sgd_cmx.pkl", 'wb')
pkl.dump(conf_mx, fileObject)
fileObject.close()

In [31]:
end = time.time()

In [32]:
print(start - end)

-608.3053050041199
